# High Throughput Screening Assay Profiling for Large Chemical Databases

This Jupyter Notebook outlines the steps needs to use PubChem's PUG-REST web service to obtain the High-throughput Screening (HTS) results for a set of compounds.

## Instructions

### Inputs

All user-input required parameters are defined in the first cell. First, `data_file` is the name of the file containing the CIDs of compounds you would like to bioprofile. This should be a single-column txt file containing the PubChem CID's of the compounds you wish to obtain HTS data for.  If you have compounds as another identifier (e.g., SMILES) PubChem offers a helpful batch-conversion tool located [here](https://pubchem.ncbi.nlm.nih.gov/idexchange/idexchange.cgi).  Second, the parameter `min_actives` specifies the number of minimum active Bioactivity Outcome responses a particular PubChem Bioassay should have to be included in the modeling matrix.

### Running the notebook

To run the notebook, in the tool bar above, select the "Cell" menu tab and select "Run All". 


### Outputs

With default parameters, two output files will be created in the directory of this Jupyer Notebook.  First, `bioprofile_long.csv` contains the bioprofile in "long" format (e.g., each row a unique, CID/AID/Bioactivity Outcome) allong with Bioassay metadata (e.g., Bioassay names, descriptions, targets, etc).  The second file, `bioprofile_matrix.csv` is a file suitable for modeing.  E.g., wide/ matrix format with duplicated Bioactivity Outcomes are merged, and Bioactivity Outcomes are integer encoded (Active/Probe outcomes are converted to 1, Inactive to -1 and Missing/Inconclusive/Unspecified data as 0).   

In [ ]:
# Change the data file as needed
# should be a single column list
# consisting of cids.

data_file = 'data/cids.txt'
min_actives = 10

In [ ]:
import requests
import pandas as pd
from itertools import zip_longest


def bioassay_post(identifier_list, identifier='cid', output='csv'):
    """ uses PubChem's PUG-Rest service to obtain assay summaries for a target set of chemicals 
    via a POST request.
    
    identifer_list: a list of chemical identifiers, the type of identifier should match with that outlined
    in the 'identifer' arguement with the default being PubChem Compound Identifier (CID).
    
    identifier: the chemical identifier (e.g., cid, smiles, etc.) of the chemicals in 'identifier_list'. Can be any chemical identifier 
    as outlined in the PubChem PUG-Rest documentation.  Default=cid
    
    output: the output format (e.g., csv, json, etc.) of the 
    
    """
    
    # convert list of identifers to str
    identifier_list = list(map(str, identifier_list))

    # make the base URL for the PubChem POST Request
    url = 'https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/{}/assaysummary/{}'.format(identifier, output)

    # encoded_list = [requests.utils.quote(s) for s in identifier_list]

    # encoded_list = requests.utils.quote(','.join(identifier_list))
    headers = {'Content-Type': 'multipart/form-data'}
    data = {identifier: ','.join(identifier_list)}

    response = requests.post(url, data=data)

    return response

def grouper(iterable, n, fillvalue=None):
    """ support function for bioprofile, used to create n equaled-sized
    sets from an iterable.
    """
    
    args = [iter(iterable)] * n
    return zip_longest(*args, fillvalue=fillvalue)


def bioprofile(identifier_list, identifier='cid', outfile='bioprofile.csv', chunk=False):

    num_compounds = len(identifier_list)
    
    # check to see whether
    # the list should be queried
    # in chunks of data or 
    # processed as a whole
    if not chunk:
        chunk_size = num_compounds
    else:
        chunk_size = chunk
    
    counter = 0

    f = open(outfile, 'w')
    header_written = False

    for gp in grouper(identifier_list, chunk_size):
        batch = [cid for cid in gp if cid]
        response = bioassay_post(batch, identifier=identifier, output='csv')

        if response.status_code == 200:
            text = response.text
            if not header_written:
                f.write(text)
                header_written = True
            else:
                header = text.split('\n')[0]
                text = text.replace(header, '')
                f.write(text)
            counter = counter + len(batch)
            print("Retrieved data for {} out of {} compounds.".format(counter, num_compounds))
        else:
            f.close()
            os.remove('data/assay_data.csv')
            print("Error: {}".format(response.status_code))
            return
    f.close()

    
def make_matrix(data_file, min_actives=0, merge_dups_by='max', outfile='bioprofile_matrix.csv'):
    """ will turn assay data file into wide (i.e., a matrix) format but performs all filtering in
        long format to save RAM"""

    df = pd.read_csv(data_file, usecols=['AID', 'CID', 'Bioactivity Outcome'])
    df['Bioactivity Outcome'] = df['Bioactivity Outcome'] \
                                    .replace('Inactive', -1) \
                                    .replace('Active', 1) \
                                    .replace('Probe', 1) \
                                    .replace('Inconclusive', 0) \
                                    .replace('Unspecified', 0) \
                                    .fillna(0)

    df['Activity Transformed'] = df.groupby(['CID', 'AID'])['Bioactivity Outcome'].transform(merge_dups_by)
    
    # for the num actives count, a CID is considered active
    # for a given AID if it has an active response for any of its
    # bioactivity outcomes for that AID.
    df['Bioactivity Outcome Max'] = df.groupby(['CID', 'AID'])['Bioactivity Outcome'].transform('max')
    
    # take only one response for a 
    # CID/AID pair, ie., the transformed
    # bioactivity value  
    df = df.drop_duplicates(['CID', 'AID', 'Activity Transformed'])
    # CID/AID/Bioactivity Outcome should be unique
    # just like CID/AID/Bioactivity Outcome Maz
    df_tmp = df.groupby('AID')['Bioactivity Outcome Max'].apply(lambda x: (x == 1).sum()).reset_index(name='Num_Active')
    df = df.merge(df_tmp)
    df = df[df.Num_Active >= min_actives]
    


    # turn into wide format
    matrix = df.pivot(index='CID', columns='AID', values='Activity Transformed').fillna(0)
    
    matrix.to_csv(outfile)

In [ ]:
# open the csv file and read
# store identifiers as a list
target_compounds = []

csv_file = open(data_file, "r")

for line in csv_file:
    cmp = line.strip()
    target_compounds.append(cmp)

csv_file.close()

In [ ]:
# collects data in 'long' format
# and writes to a csv file specified
# in the outfile parameter
bioprofile(target_compounds, chunk=100, outfile='bioprofile_long.csv')

In [ ]:
# convert to a matrix that is
# more suitable for modeling
make_matrix('bioprofile_long.csv', min_actives=min_actives, outfile='bioprofile_matrix.csv')